In [1]:
import os
import pandas as pd
import glob
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import urllib

In [12]:
selection = 'marketinsights'
directory = "./utils/data/Scraped News/"
file_pattern = f"{selection}_data_*.csv"
files = glob.glob(os.path.join(directory, file_pattern))
print(files)
latest_file = max(files, key=os.path.getctime)
print(latest_file)

['./utils/data/Scraped News/marketinsights_data_2024-07-01.csv']
./utils/data/Scraped News/marketinsights_data_2024-07-01.csv


In [182]:
from utils.marketinsights import safe_literal_eval, process_tables
df = pd.read_csv(latest_file)
df.head()


,title,link,ticker,date,source,Article content,raw,tables,Time,Executives,Shareholders
0,Cordiant Digital Infrastructure Limited Announ...,https://www.marketscreener.com/quote/stock/COR...,CORD,02:00am,S&P Capital IQ,Cordiant Digital Infrastructure Limited announ...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'David Charles B...,2024-07-01 02:00:00,"[['David Charles Bonar', 'Treasurer', '53', '2...","[['Newton Investment Management Ltd.', '37,855..."
1,Certain Common Stock of Public Joint-Stock Com...,https://www.marketscreener.com/quote/stock/SOV...,SVCB,2024-06-30,S&P Capital IQ,Certain Common Stock of Public Joint-Stock Com...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Dmitry Vladimir...,2024-06-30 00:00:00,"[['Aleksey Valeryevich Panferov', 'Corporate O...",[]
2,Certain A Shares of Chengdu Qinchuan IoT Techn...,https://www.marketscreener.com/quote/stock/CHE...,688528,2024-06-30,S&P Capital IQ,Certain A Shares of Chengdu Qinchuan IoT Techn...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Ze Hua Shao', 1...",2024-06-30 00:00:00,"[['Jing Zhang', 'Chief Administrative Officer'...","[['Gfund Management Co., Ltd.', '1,454,100', '..."
3,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,https://www.marketscreener.com/quote/stock/ARA...,2222,Jun. 30,Reuters,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Amin Al-Nasser'...,2024-06-30 00:00:00,"[['Ahmad Al-Khowaiter', 'Chief Tech/Sci/R&D Of...","[['Government of Saudi Arabia', '198,890,120,0..."
4,Certain Employee Stock Options of Corebridge F...,https://www.marketscreener.com/quote/stock/COR...,CRBG,Jun. 29,S&P Capital IQ,Certain Employee Stock Options of Corebridge F...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Kevin Hogan', 1...",2024-06-29 00:00:00,"[['Adam Burk', 'Director/Board Member', '47', ...","[['AMERICAN INTERNATIONAL GROUP, INC.', '294,2..."


In [254]:

df['Industry'] = df['raw'].apply(get_industry)
df.head()

,title,link,ticker,date,source,Article content,raw,tables,Time,Executives,Shareholders,Industry,Contact Information,Country_phone,Country_city,Country,Country_candidates
0,Cordiant Digital Infrastructure Limited Announ...,https://www.marketscreener.com/quote/stock/COR...,CORD,02:00am,S&P Capital IQ,Cordiant Digital Infrastructure Limited announ...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'David Charles B...,2024-07-01 02:00:00,"[['David Charles Bonar', 'Treasurer', '53', '2...","[['Newton Investment Management Ltd.', '37,855...",Financials,{'Company Name': 'Cordiant Digital Infrastruct...,"{United States, Canada}",{Guernsey},Guernsey,"{United States, Guernsey, Canada}"
1,Certain Common Stock of Public Joint-Stock Com...,https://www.marketscreener.com/quote/stock/SOV...,SVCB,2024-06-30,S&P Capital IQ,Certain Common Stock of Public Joint-Stock Com...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Dmitry Vladimir...,2024-06-30 00:00:00,"[['Aleksey Valeryevich Panferov', 'Corporate O...",[],Financials,"{'Company Name': 'Sovcombank PJSC', 'Address L...","{Kazakhstan, Russia}",{Russia},Russia,{Russia}
2,Certain A Shares of Chengdu Qinchuan IoT Techn...,https://www.marketscreener.com/quote/stock/CHE...,688528,2024-06-30,S&P Capital IQ,Certain A Shares of Chengdu Qinchuan IoT Techn...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Ze Hua Shao', 1...",2024-06-30 00:00:00,"[['Jing Zhang', 'Chief Administrative Officer'...","[['Gfund Management Co., Ltd.', '1,454,100', '...",Industrials,{'Company Name': 'Chengdu Qinchuan IoT Technol...,{Unknown},{China},China,{China}
3,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,https://www.marketscreener.com/quote/stock/ARA...,2222,Jun. 30,Reuters,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Amin Al-Nasser'...,2024-06-30 00:00:00,"[['Ahmad Al-Khowaiter', 'Chief Tech/Sci/R&D Of...","[['Government of Saudi Arabia', '198,890,120,0...",Energy,"{'Company Name': 'Saudi Arabian Oil Co.', 'Add...",{Unknown},{Saudi Arabia},Saudi Arabia,{Saudi Arabia}
4,Certain Employee Stock Options of Corebridge F...,https://www.marketscreener.com/quote/stock/COR...,CRBG,Jun. 29,S&P Capital IQ,Certain Employee Stock Options of Corebridge F...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Kevin Hogan', 1...",2024-06-29 00:00:00,"[['Adam Burk', 'Director/Board Member', '47', ...","[['AMERICAN INTERNATIONAL GROUP, INC.', '294,2...",Financials,"{'Company Name': 'Corebridge Financial, Inc.',...","{United States, Canada}",{United States},United States,{United States}


In [249]:
import re
import unicodedata
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import cosine

def get_industry(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Locate all card-header divs
    card_headers = soup.find_all('div', class_='card-header')

    # Iterate through each card-header div and extract the required information
    for card_header in card_headers:
        # Get the title text from the card-header div
        title_text = card_header.get_text(strip=True)
        if title_text == 'Sector':
            # Find the next sibling card-content div
            card_content = card_header.find_next_sibling('div', class_='card-content')
            
            if card_content:
                # Get the sector details from the card-content div
                sector_details = list(map(lambda x: x.strip(), card_content.get_text().split('\n')))
                for sector in sector_details:
                    if sector:
                        return sector
    return 'Unknown'

def get_contact_information(html_content):
    result = {}
    soup = BeautifulSoup(html_content, 'html.parser')
    # Find the div with class 'card-content'
    contact_info_div = soup.find_all('div', class_ = 'card mb-15')
    for div in contact_info_div:
        try:
        # Extract the paragraphs within this div
            contact_info_paragraphs = div.find_all('p', class_='m-0')
            if contact_info_paragraphs:
                
                company_name = contact_info_paragraphs[0].text
                address_line_1 = contact_info_paragraphs[1].text
                address_line_2 = contact_info_paragraphs[2].text
                phone_number = contact_info_paragraphs[3].text

                # Extract the website URL
                website = div.find('a', class_='m-0')['href']

            # Print the extracted information
            result['Company Name'] = company_name
            result['Address Line 1'] = address_line_1
            result['Address Line 2'] = address_line_2
            result['Phone Number'] = phone_number
            result['Website'] = website
        except:
            pass
    return result

def get_phone_mapping():
    url = "https://www.countrycode.org"
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table
    table = soup.find('table')

    # Extract the table headers
    headers = []
    for th in table.find_all('th'):
        headers.append(th.text.strip())

    # Extract the table rows
    rows = []
    for tr in table.find_all('tr'):
        cells = tr.find_all('td')
        row = [cell.text.strip() for cell in cells]
        if row:
            rows.append(row)

    # Create a DataFrame
    df = pd.DataFrame(rows, columns=headers)[['COUNTRY', 'COUNTRY CODE']]

    storage = {}
    for i, row in df.iterrows():
        if row['COUNTRY CODE'] not in storage.keys():
            storage[row['COUNTRY CODE']] = {row['COUNTRY']}
        else:
            storage[row['COUNTRY CODE']].add(row['COUNTRY'])
    return storage

def label_country_by_phone(contact_information, storage):
    country = {'Unknown'}
    try:
        phone_number = contact_information['Phone Number'].replace('-', ' ').replace('+', '').replace('(', '').replace(')', '')
        extension = phone_number.split(' ')[0]
        try:
            country = storage[extension]
        except:
            try:
                country = storage[f'1-{extension}']
            except:
                if extension:
                    country = {'United States','Canada'}
                else:
                    country = {'Unknown'}
    except:
        country = {'Unknown'}
    return country

def remove_diacritics(input_str):
    # Normalize the string to decompose diacritics
    normalized_str = unicodedata.normalize('NFD', input_str)
    
    # Filter out the diacritic marks
    non_diacritic_str = ''.join(
        char for char in normalized_str if unicodedata.category(char) != 'Mn'
    )
    return non_diacritic_str

def get_city_mapping():
    cities = pd.read_csv('worldcities.csv')
    city_storage = {}
    city_storage['St. Peter Port'] = {'Guernsey'}
    for i, row in cities.iterrows():
        city = remove_diacritics(row['city'])
        if city not in city_storage.keys():
            city_storage[city] = {row['country']}
        else:
            city_storage[city].add(row['country'])
    return city_storage

def extract_city_names(lst):
    for i in range(len(lst) - 1, -1, -1):
        if re.search(r'\d', lst[i]):
            if i + 1 < len(lst):
                return ' '.join(lst[i+1:]).split(',')[-1].strip()

def label_country_by_city(contact_information, city_storage):
    country = {'Unknown'}
    try:
        temp = contact_information['Address Line 2'].split(' ')
        city = extract_city_names(temp)
        country = city_storage[city]
    except:
        country = {'Unknown'}
    return country

def get_intersection(a, b):
    if a.intersection(b):
        return a.intersection(b)
    else:
        if 'Unknown' in a:
            if b:
                if 'Unknown' in b:
                    return 'Unknown'
                else:
                    return b
            else:
                return 'Unknown'
        else:
            if 'Unknown' in b:
                return a
            else:
                c = a | b
                return c

# Function to get embeddings
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Function to find the most similar country
def find_country(address, countries):
    country_embeddings = get_embeddings(countries)
    address_embedding = get_embeddings([address])
    address_embedding = address_embedding.flatten() # Convert to 1-D array
    similarities = [1 - cosine(address_embedding, country_embedding) for country_embedding in country_embeddings]
    most_similar_country = countries[similarities.index(max(similarities))]
    return most_similar_country

def final_country(contact_information, candidates):
    if candidates:
        if len(candidates) == 1:
            return list(candidates)[0]
        else:
            if contact_information:
                temp = contact_information['Address Line 2'].split(' ')
                city = extract_city_names(temp)
                return find_country(city, list(candidates))
            else:
                return 'Unknown'
    else:
        return 'Unknown'

phone_storage = get_phone_mapping()
city_storage = get_city_mapping()

model_name = 'nomic-ai/nomic-embed-text-v1'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

df['Industry'] = df['raw'].apply(get_industry)
df['Contact Information'] = df['raw'].apply(get_contact_information)
df['Country_phone'] = df['Contact Information'].apply(lambda x: label_country_by_phone(x, phone_storage))
df['Country_city'] = df['Contact Information'].apply(lambda x: label_country_by_city(x, city_storage))
df['Country_candidates'] = df.apply(lambda x: get_intersection(x.Country_phone, x.Country_city), axis = 1)
df['Country'] = df.apply(lambda x: final_country(x['Contact Information'], x.Country_candidates), axis = 1)
df.to_csv('marketinsights_data_2024-07-02.csv', index = False)


<All keys matched successfully>


In [250]:
df

,title,link,ticker,date,source,Article content,raw,tables,Time,Executives,Shareholders,Industry,Contact Information,Country_phone,Country_city,Country,Country_candidates
0,Cordiant Digital Infrastructure Limited Announ...,https://www.marketscreener.com/quote/stock/COR...,CORD,02:00am,S&P Capital IQ,Cordiant Digital Infrastructure Limited announ...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'David Charles B...,2024-07-01 02:00:00,"[['David Charles Bonar', 'Treasurer', '53', '2...","[['Newton Investment Management Ltd.', '37,855...",Closed End Funds,{'Company Name': 'Cordiant Digital Infrastruct...,"{United States, Canada}",{Guernsey},Guernsey,"{United States, Guernsey, Canada}"
1,Certain Common Stock of Public Joint-Stock Com...,https://www.marketscreener.com/quote/stock/SOV...,SVCB,2024-06-30,S&P Capital IQ,Certain Common Stock of Public Joint-Stock Com...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Dmitry Vladimir...,2024-06-30 00:00:00,"[['Aleksey Valeryevich Panferov', 'Corporate O...",[],Other Banks,"{'Company Name': 'Sovcombank PJSC', 'Address L...","{Kazakhstan, Russia}",{Russia},Russia,{Russia}
2,Certain A Shares of Chengdu Qinchuan IoT Techn...,https://www.marketscreener.com/quote/stock/CHE...,688528,2024-06-30,S&P Capital IQ,Certain A Shares of Chengdu Qinchuan IoT Techn...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Ze Hua Shao', 1...",2024-06-30 00:00:00,"[['Jing Zhang', 'Chief Administrative Officer'...","[['Gfund Management Co., Ltd.', '1,454,100', '...",Testing & Measuring Equipment,{'Company Name': 'Chengdu Qinchuan IoT Technol...,{Unknown},{China},China,{China}
3,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,https://www.marketscreener.com/quote/stock/ARA...,2222,Jun. 30,Reuters,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Amin Al-Nasser'...,2024-06-30 00:00:00,"[['Ahmad Al-Khowaiter', 'Chief Tech/Sci/R&D Of...","[['Government of Saudi Arabia', '198,890,120,0...",Integrated Oil & Gas,"{'Company Name': 'Saudi Arabian Oil Co.', 'Add...",{Unknown},{Saudi Arabia},Saudi Arabia,{Saudi Arabia}
4,Certain Employee Stock Options of Corebridge F...,https://www.marketscreener.com/quote/stock/COR...,CRBG,Jun. 29,S&P Capital IQ,Certain Employee Stock Options of Corebridge F...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Kevin Hogan', 1...",2024-06-29 00:00:00,"[['Adam Burk', 'Director/Board Member', '47', ...","[['AMERICAN INTERNATIONAL GROUP, INC.', '294,2...",Other Life & Health Insurance,"{'Company Name': 'Corebridge Financial, Inc.',...","{United States, Canada}",{United States},United States,{United States}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Hitachi Reportedly Planning to Sell its 40% St...,https://www.marketscreener.com/quote/stock/JOH...,JCI,Jun. 17,S&P Capital IQ,"Hitachi, Ltd. (TSE:6501) is planning to sell i...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'George Oliver',...",2024-06-17 00:00:00,"[['Ayesha Khanna', 'Director/Board Member', '5...","[['Dodge & Cox', '79,938,324', '11.87%', '5 74...","Heating, Ventilation & Air Conditioning Systems",{'Company Name': 'Johnson Controls Internation...,{Ireland},{Ireland},Ireland,{Ireland}
146,"China Tungsten And Hightech Materials Co.,Ltd ...",https://www.marketscreener.com/quote/stock/CHI...,000657,Jun. 17,S&P Capital IQ,"China Tungsten And Hightech Materials Co.,Ltd ...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Kang De Xie', 1...",2024-06-17 00:00:00,"[['Chang Long Xu', 'Director/Board Member', '6...",[],Other Specialty Mining & Metals,{'Company Name': 'China Tungsten & Hightech Ma...,{China},{China},China,{China}
147,MAHB Reportedly Aborts Plan to Sell Istanbul A...,https://www.mark

In [224]:
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import cosine

# Load pre-trained model and tokenizer
model_name = 'nomic-ai/nomic-embed-text-v1'
#model_name = 'intfloat/multilingual-e5-large-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# Function to get embeddings
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Function to find the most similar country
def find_country(address, countries):
    country_embeddings = get_embeddings(countries)
    address_embedding = get_embeddings([address])
    address_embedding = address_embedding.flatten() # Convert to 1-D array
    similarities = [1 - cosine(address_embedding, country_embedding) for country_embedding in country_embeddings]
    most_similar_country = countries[similarities.index(max(similarities))]
    return most_similar_country

<All keys matched successfully>


In [239]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Load pre-trained model and tokenizer
model_name ='intfloat/multilingual-e5-large-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# Function to get embeddings
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Function to find the most similar country
def find_country(address, countries):
    country_embeddings = get_embeddings(countries)
    address_embedding = get_embeddings([address])
    address_embedding = address_embedding.flatten() # Convert to 1-D array
    similarities = [1 - cosine(address_embedding, country_embedding) for country_embedding in country_embeddings]
    most_similar_country = countries[similarities.index(max(similarities))]
    return most_similar_country

# Example usage
address = "Boston"
countries = ["United States", "Canada", "Philippines"]
print(find_country(address, countries))

United States


In [ ]:
from sentence_transformers import SentenceTransformer

# Load https://huggingface.co/sentence-transformers/all-mpnet-base-v2
model = SentenceTransformer("all-mpnet-base-v2")


# Function to get embeddings
def get_embeddings(texts):
    return model.encode(texts)

# Function to find the most similar country
def find_country(address, countries):
    country_embeddings = get_embeddings(countries)
    address_embedding = get_embeddings(address)
    similarities = model.similarity(address_embedding, country_embeddings)
    most_similar_country = countries[similarities.index(max(similarities))]
    return most_similar_country

def final_country(contact_information, candidates):
    if candidates:
        if len(candidates) == 1:
            return list(candidates)[0]
        else:
            if contact_information:
                temp = contact_information['Address Line 2'].split(' ')
                city = extract_city_names(temp)
                return find_country(city, list(candidates))
            else:
                return 'Unknown'
    else:
        return 'Unknown'

df['Country_candidates'] = df.apply(lambda x: get_intersection(x.Country_phone, x.Country_city), axis = 1)
df['Country'] = df.apply(lambda x: final_country(x['Contact Information'], x.Country_candidates), axis = 1)